# Openslide: experiment notebook

In [ ]:
# REFERENCES:
# https://github.com/bnsreenu/python_for_microscopists/blob/master/266_openslide_for_whole_slide_images/openslide_library_for_whole_slide_images.py

In [ ]:
import numpy as np
import openslide
from openslide.deepzoom import DeepZoomGenerator

# Open the TIFF image
slide = openslide.OpenSlide("../data/images/01lRJdxbiV_a.tif")

In [ ]:
# Playing with some interesting attributes
print("dimensions:", slide.dimensions)
print("nb dimensions:", len(slide.level_dimensions))  # dim at each dim
print("level_dimensions:", slide.level_dimensions)  # dim at each dim
print("level_downsamples:", slide.level_downsamples)  # downsampling ratio

In [ ]:
# Thumbnail
thumbnail = slide.get_thumbnail(size=(500, 500))

# We can convert it to numpy array
arr_thumbnail = np.array(thumbnail)

print(arr_thumbnail.shape)
thumbnail

In [ ]:
# Read a region
location = (0, 0)  # top left pixel
level = 6  # zoom
size = (1024, 1024)  # height, width

slide.read_region(location, level, size)

In [ ]:
# Generate object for tiles using the DeepZoomGenerator
TILE_SIZE = 1024

tiles = DeepZoomGenerator(slide, tile_size=TILE_SIZE, overlap=0, limit_bounds=False)

In [ ]:
print("The number of levels in the tiles object are: ", tiles.level_count)

In [ ]:
# Dimensions of the tile (tile size) for a specific tile from a specific layer
print(tiles.get_tile_dimensions(level=0, address=(0, 0)))
print(tiles.get_tile_dimensions(level=5, address=(0, 0)))
print(tiles.get_tile_dimensions(level=8, address=(0, 0)))
print(tiles.get_tile_dimensions(level=11, address=(0, 0)))
print(tiles.get_tile_dimensions(level=18, address=(0, 0)))

In [ ]:
MAX_RES_LEVEL = tiles.level_count - 1

In [ ]:
# nb tiles at the best resolution level (the last one) * tile size ~= dim images (depends on limit bounds)
print("slide dimensions:", slide.dimensions)
print(
    "nb tiles at the best resolution level:", (tiles.level_tiles[tiles.level_count - 1])
)
print(
    "nb tiles at the best resolution level (the last one) * tile size:",
    (
        tiles.level_tiles[MAX_RES_LEVEL][0] * TILE_SIZE,
        tiles.level_tiles[MAX_RES_LEVEL][1] * TILE_SIZE,
    ),
)


In [ ]:
# Get a tile at a (layer, adress), i.e. (zoom, (x, y))
tiles.get_tile(level=MAX_RES_LEVEL, address=(25, 34))

In [ ]:
cols, rows = tiles.level_tiles[MAX_RES_LEVEL]
print("nb of tiles:", (rows, cols))

# Experiment zone to debug misfiltered tiles

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import openslide
from openslide.deepzoom import DeepZoomGenerator
from digilut.tiles.filter_tiles import (
    has_too_many_black,
    has_too_many_white,
    is_green_weaker,
    is_not_tissue,
)

# Open the TIFF image
slide = openslide.OpenSlide("../data/images/01lRJdxbiV_a.tif")

# Generate object for tiles using the DeepZoomGenerator
TILE_SIZE = 1024
MAX_RES_LEVEL = 18

tiles = DeepZoomGenerator(slide, tile_size=TILE_SIZE, overlap=0, limit_bounds=False)


tile = tiles.get_tile(level=MAX_RES_LEVEL, address=(16, 141))
tile
tile_arr = np.array(tile)
tile_arr.shape

hist = np.array(tile.convert("L")).flatten()

plt.figure()
b, bins, patches = plt.hist(hist, 255)
plt.xlim([0, 255])
plt.show()

removed = is_not_tissue(tile)
arr_tile = np.array(tile.convert("L"))
arr_tile_rgb = np.array(tile.convert("RGB"))

print("too much white:", has_too_many_white(arr_tile))
print("green weaker:", not is_green_weaker(arr_tile_rgb))
print("too much black:", has_too_many_black(arr_tile))

print("Classified as background:", removed)

tile

In [ ]:
arr_tile_rgb = np.array(tile.convert("RGB"))

arr_tile_rgb.mean(axis=tuple(range(arr_tile_rgb.ndim - 1)))